In [1]:
import os

In [2]:
# check current working directory
%pwd

'g:\\Jagadish\\Production-Ready-Chest-Cancer-Detection-Deep-Learning-Model-MLOps-with-MLflow-DVC-CI-CD-and-AWS\\research'

In [3]:
# change directory to root directory of project for importing files
os.chdir("../")

In [4]:
# checking current working directory again after change
%pwd

'g:\\Jagadish\\Production-Ready-Chest-Cancer-Detection-Deep-Learning-Model-MLOps-with-MLflow-DVC-CI-CD-and-AWS'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

[2024-12-12 18:25:08,386: INFO: __init__: Logging setup is complete.]


In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [8]:
import os
import gdown
import zipfile

from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

In [9]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
     
    def download_file(self)-> str:
        '''
        Fetch data from the url
        '''

        try: 
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id,zip_download_dir)

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise e
        
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [10]:
try:
    # Initialize configuration manager and get data ingestion configuration
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    
    # Initialize data ingestion object with the configuration
    data_ingestion = DataIngestion(config=data_ingestion_config)
    
    # Perform the file download and extraction
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
    
except Exception as e:
    # Log the exception and raise it
    logger.error(f"An error occurred during the data ingestion process: {e}")
    raise e


[2024-12-12 18:25:10,468: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-12-12 18:25:10,469: INFO: common: yaml file: params.yaml loaded successfully]
[2024-12-12 18:25:10,469: INFO: common: created directory at: artifacts]
[2024-12-12 18:25:10,484: INFO: common: created directory at: artifacts/data_ingestion]
[2024-12-12 18:25:10,488: INFO: 1983665874: Downloading data from https://drive.google.com/file/d/1Vcg53XC4JCBMxwCsOke2o_k9J1czV_hW/view?usp=sharing into file artifacts/data_ingestion/data.zip]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1Vcg53XC4JCBMxwCsOke2o_k9J1czV_hW
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1Vcg53XC4JCBMxwCsOke2o_k9J1czV_hW&confirm=t&uuid=c562516a-37c9-4a11-be18-d9401755d073
To: g:\Jagadish\Production-Ready-Chest-Cancer-Detection-Deep-Learning-Model-MLOps-with-MLflow-DVC-CI-CD-and-AWS\artifacts\data_ingestion\data.zip
100%|██████████| 49.0M/49.0M [00:10<00:00, 4.75MB/s]

[2024-12-12 18:25:25,512: INFO: 1983665874: Downloaded data from https://drive.google.com/file/d/1Vcg53XC4JCBMxwCsOke2o_k9J1czV_hW/view?usp=sharing into file artifacts/data_ingestion/data.zip]
